<a href="https://colab.research.google.com/github/Charlee0616/Data-Mining/blob/main/C%26W_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import math
from scipy.stats import norm

In [2]:
penguins = pd.read_csv("https://github.com/mbrudd/csci290/raw/refs/heads/main/data/penguins.csv")

In [3]:
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [4]:
def Naive_Bayes_Classifier(X, y, instance, numerical, categorical):

  priors = {}
  likelihoods = {}


  for clss in X[y].unique():
    data = X[X[y] == clss] # Create a dataset for the rows where the clss is True
    priors[clss] = len(data) / len(X) # Calculate the priors for the dataset it created
    likelihoods[clss] = {} # Makes a dictionary where the clss is the key for the likelihoods
    for cat in categorical: # Gets likelihoods for categorical features
      likelihoods[clss][cat] = data[cat].value_counts(normalize=True).to_dict()
    for num in numerical: # Gets likelihoods for numerical features
      likelihoods[clss][num] = norm( loc=data[num].mean(), scale = data[num].std())

    posterior = {} # Create a dictionary for calculating the posteriors

    for clss in priors:
      post = priors[clss] # Sets post equal to priors for that clss
      for cat in categorical: # Categorical features
        if cat in instance: # Checks if the cat is in the unseen instance
          if instance[cat] in likelihoods[clss][cat]:
            post *= likelihoods[clss][cat][instance[cat]] # Multiplies post by the likelihoods for the cat features and then sets the post equal to that
          else:
            post *= 0
        else:
          pass
      for num in numerical: # Numerical features
        post *= likelihoods[clss][num].pdf(instance[num]) # Multiplies post by the likelihoods for the num features and then sets the post equal to that

      posterior[clss] = post # Sets the posterior for that clss equal to the post

    return max(posterior, key=posterior.get)

In [5]:
# Dataset
df = pd.read_csv("https://github.com/mbrudd/csci290/raw/refs/heads/main/data/penguins.csv") # Change this to new dataset
df.head()
# Tagret column in the dataset
target = "species"
# Unseen Instance
instance = {'bill_length_mm': 38.2, 'bill_depth_mm': 48.1, 'flipper_length_mm': 180.0, 'body_mass_g': 3800, 'sex': 'male'}  # Make new unseen instance
# Numerical features
numerical = []
# Categorical features
categorical = []

## Remove [2:-1] if you want to test on all columns ##
for attrib in df.columns[2:-1]:
  if df[attrib].dtype in ['float64', 'int64']:
    # Considers the column numerical if there are more than 10 unique values
    if len( df[attrib].unique() ) > 10:
      numerical.append(attrib)
  else:
    categorical.append(attrib)

Naive_Bayes_Classifier(df, target, instance, numerical, categorical)


'Adelie'

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [40]:
df.dropna(inplace=True)
data = pd.get_dummies(df, columns = ['species', 'island', 'sex'], drop_first=True)

In [42]:
X = df.drop(target, axis=1)
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
model = GaussianNB()  # Choose the appropriate Naive Bayes variant
model.fit(X_train, y_train)

ValueError: could not convert string to float: 'Biscoe'

In [44]:
y_pred = model.predict(X_test)

ValueError: could not convert string to float: 'Dream'

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

print('Classification Report:')
print(classification_report(y_test, y_pred))

print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))